# CH 17 and 18

## TOC<a id='toc'></a>
* [Ch17 Notes](#ch17_notes)
* [Ch18 Notes](#ch18_notes)

### CH17 Notes <a id='ch17_notes'></a>
[toc](#toc)
### Concurrency

* the `requests` library by Keneth Reitz is more powerful and easier to use than urllib.request module from python 3 std library
    * is considered a model Pythonic API

## Downloading with concurrent.futures
* main features are `ThreadPoolExecutor` and `ProcessPoolExecutor` classes
    - implement interfaces that allow you to submit callables for execution in different threads or processes respectively
    - they manage an internal pool of threads or processes and a queue of tasks to be exectuted.

example code:
```
def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
        
       return len(list(res))
```

* `executor.__exit__` method will call `executor.shutdown(wait=True)`, which will block untill all threads are done.
* map function returns a generator that can be iterated over to retrieve the value returned by each function.
    - if any function raised an exception, the exception would be raised when calling next on that generator
* very often, body of the loop is refactored into a separate function when calling either sequentially or concurrently.

## What are Futures?
* as of python 3.4 two Futures classes: `concurrent.futures.Futures` and `asyncio.Future`
       - an instance of either class represents a deferred computation that may or may not have completed.
       - their state of completion can be queried, and their results (or exceptions) can be retrieved when avaialble.
* You and I should not create them! They should be instantiated exclusively by concurrency framework.
    - they are instantiated when some work is scheduled - this typically returns a future.
* both have a .done() method to check if it is done - but usually you use the `.add_done_callback()` and provide a callable instead.
* there is also a `.result()` method, which if done, returns result, or re-reaises any exceptions. 
    - if future is not done, result behavior is very different for the two future classes: concurrency.future blocks and has optional timeout, asyncio does not support timeout, and preferred is to use yield from - which doesn't work with concurrency.
* Executor.map returns an iterator in which `__next__` calls the `result` method of each future.
* concurrent.futures has an .as_completed() function that takes an iterable of futures and returns an iterator that yiedls futures as they are done.
    - they are yielded in whatever order they finish.

## Blocking I/O and the GIL
* The CPython interpreter is not thread-safe internally, so it has a Global Interpreted Lock (GIL), which allows only one thread at a time to executed python bytecodes. That's why a single python processs usually cannot use multiple CPU cores at the same time.
    - when we write python code, we have no control over the GIL, but a built-in function, or an extension written in C can release the GIL. (This complciates the code of the library considerably, so most authors don't do it.)
* however all standard libray functions that perform blockin I/O release the GIL when waiting for a result from the OS. So while one Python thread is waiting for a response from the network (or other?), the blocked I/O function releases the GIL so another thread can run.
    - time.sleep() function also releases the GIL.

## ProcessPoolExecutor
* truly parallel computation - bypasses GIL and leverages all available CPUs
* won't help much if the work is I/O bound, where much of the time is passed waiting - threads are better for this
    - not limited by number od CPUs
* If you are using CPU-intensive work in python, you should try PyPy - it is much faster.

* Executor.map is easy to use, but it returns results in the same order as the calls are started; so your code might block waiting for a result, while other results further down the call line have already completed.
* to avoid this use `Executor.submit` and `futures.as_completed`
    - this is also more flexible because you can use different callables and arguments

## Downloads with Progress Display and Error Handling

In [4]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
    time.sleep(0.01)

100%|██████████| 1000/1000 [00:10<00:00, 95.28it/s]


* can handle some error in the callable; can handle other errors in the function that creates the executor.

* Usually when using `futures.as_completed` it is useful to build a dcit mapping each future to other data that may be useful when future is completed (in the flag example, the future is mapped to the country code)
    - makes it easy to do a follow up process despite the fact that they are out of order

## Threading and multiprocessing alternatives
* originally had `thread` module; since python 3 it was deprecated in favor of a higher level `threading` module
    - original renamed to `_thread` to make obvious it is just low-level implementation detail and shouldn't be used
* futures.ThreadPoolExecutor is a high level access for this, but if it doesn't meet your needs, you might need to build your own solution from `threading`
* For CPU_bound work, sidestep GIL with `futures.ProcessPoolExecutor`. But if your case is too complex, you need more advance tools like `multiprocessing`
    - `multiprocessing` also offers facilities to solve the biggest challenge faced by collaborating processes: *(how to pass data around*

Best advice: **Don't try to manage threads and locks** - let system programmers that know what they are doing do that; you should just use what they have built.

- of course, these are generally designed for simple, so called **embarassingly parallel** jobs in mind, but that is typically what you need.
- as opposed to operating systems or database servers

Even for non embarrisingly simple, threads and locks are not the answer - use higher level abstractions!

### CH18 Notes <a id='ch18_notes'></a>
[toc](#toc)
### Concurrency with asyncio

- **Concurrency** is about dealing with lots of things at once
- **Parallelism** is about doing lots of things at once
- Not the same, but related

* A moder laptop has 4 CPU cores but is routinely running more than 100 processes at any given time under normal, causal use. So in practice, most processsing happens concurrently, not in parallel

* `asyncio` is a package that implements concurrency with coroutines driven by an event loop.
    - one of the largest and most ambitious libraries ever added to python
    - developed by the benevolent dictator himself

In [1]:
import time

In [5]:
for i in range(1,11):
    mess = str(i)
    print(mess,end='')
    time.sleep(1)
    print('\x08'*len(mess), end='')
print('Blast off!')

Blast off!


* `other_thread = threading.Thread(target=..., args=(...))` can be used to  create new threads
    - other_thread.start() kicks it off
    - typically comminunicate with it via some object which it has a reference for, and you can manipulate object in main thread.

* Asyncio is stricter about corountines: must use the **yield from** and not the **yield** in the body
* shoud be driven by caller using *yield from*, or by `asyncio.async()` function
* should be decorated with @asyncio.coroutine [not mandatory but highly advisable]
* you can communicate with ascyncio corroutines directly - `spinner.cancel()`, and don't need to communicate via shared reference to signal objec as with threading
* use `asyncio.sleep()` instead of `time.sleep()` to sleep without blocking the event loop
* ayncio uses *task* abstraction - they drive coroutines
    - similar to thread invoking callables
    - you don't instantiate them yourself, but are returned from asyncio.async(), etc.
* with threads, you must be careful to hold locks, etc. to protect from interruption - with coroutins, everything is protected from interaction by default.
    * you must explicitly yield to let the rest of the program run
    * only one corroutine runs at a time - yielding gives control back to scheduler.

trick suggested by Guido van Rossum: "squint and pretend the yield from keywords are not there" - this makes it easier to follow asyncio code

In summary: as we use `asyncio`, our asynchronous code consists of coroutines that are delegating generators friven by `asyncio` itself and that ultimately delegate to `asyncio` library coroutines - possibly by way of some third-party library such as `aiohttp`.

* If an L1 cache read took 1 sec, a network read would take something like 2.5 years. Can't halt application for that
* solutions:
    - run each blocking operation in a separate thread
    - turn each blocking operation into a nonblocking asynchronous call
* Threads work fine, but the memory overhead ofr each OS thread (the kind that python uses) is on the order of MB. Can't affor id using thousands.
* Callbacks are the traditional way to implement asynchronous calls with low memory ovehread [ME: are these not still using a bunch of OS threads?]
    - they are a low-level concept, similart to the oldest and most primitive concurrency mechanism of all: hardware interrupts
* of course, we can only make callbacks work becaue the event loop underlying our asynchronous applications can rely on infrastructures that uses interrups, theads, polling, background processes, etc. to ensure that multiple concurrent requests make progress and eventually get done.

* here use **semaphore** as throltling device
* a **semaphore** is an object that holds and internal counter that is decremented whenever we call the `.acquire()` coroutine method on it, and incremented when we call the `.release()` coroutine methods.
    - initial value of the counter is set when the Semaphore is instantiated
    - in the book he uses the semaphore as a context manager

* local filesystem access is blocking

* corroutines are a big improvement over nested callbacks for trying to synchronize asychronous coroutine, by replacing the call back with yield from 